# Adding Sentiment to LSTM

In [1]:
#navigating to properfile
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/Colab Notebooks/deeplearning/project/' #MODIFY THIS PATH FOR YOUR OWN DEVICE
import os
os.chdir(path)
os.path.abspath(os.getcwd())

Mounted at /content/drive


'/content/drive/MyDrive/Colab Notebooks/deeplearning/project'

In [2]:
import pandas as pd
import numpy as np
from math import pi
import os
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, Markdown
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from itertools import product

In [8]:
full = pd.read_csv('fine-tuned/kaggle_tweets/NBADataset1.csv')

In [6]:
train = pd.read_csv('train_outputs.csv')

In [7]:
test = pd.read_csv('test_outputs.csv')

Sentiment is on a monthly basis -> add sentiment to games on a monthly basis

In [9]:
path = '/content/drive/MyDrive/Colab Notebooks/deeplearning/project/nba_game_data' #MODIFY THIS PATH FOR YOUR OWN DEVICE
os.chdir(path)
os.path.abspath(os.getcwd())

'/content/drive/MyDrive/Colab Notebooks/deeplearning/project/nba_game_data'

In [10]:
games_details = pd.read_csv('games_details.csv')
players = pd.read_csv('players.csv')
teams = pd.read_csv('teams.csv')
ranking = pd.read_csv('ranking.csv')
games  = pd.read_csv('games.csv')

<ipython-input-10-4f7033f9cbb3>:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  games_details = pd.read_csv('games_details.csv')


In [11]:
path = '/content/drive/MyDrive/Colab Notebooks/deeplearning/project/results' #MODIFY THIS PATH FOR YOUR OWN DEVICE
os.chdir(path)
os.path.abspath(os.getcwd())

'/content/drive/MyDrive/Colab Notebooks/deeplearning/project/results'

In [14]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from keras import optimizers
from keras.models import Sequential, load_model
from keras.layers import Dense,Activation, LSTM, Dropout, TimeDistributed, Flatten, BatchNormalization
# from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

from sklearn.utils import class_weight
from math import sqrt
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

## Data Prep

In [15]:
selected_features = list(games.columns)
to_drop = ['GAME_STATUS_TEXT','TEAM_ID_home']
for var in to_drop:
  selected_features.remove(var)

In [16]:
games = games[selected_features]

In [33]:
teams.columns

Index(['LEAGUE_ID', 'TEAM_ID', 'MIN_YEAR', 'MAX_YEAR', 'ABBREVIATION',
       'NICKNAME', 'YEARFOUNDED', 'CITY', 'ARENA', 'ARENACAPACITY', 'OWNER',
       'GENERALMANAGER', 'HEADCOACH', 'DLEAGUEAFFILIATION', 'TEAM_NAME'],
      dtype='object')

In [36]:
home_teams = teams[['TEAM_ID','TEAM_NAME']]
home_teams.rename(columns={'TEAM_ID':'HOME_TEAM_ID'},inplace=True)
vis_teams = teams[['TEAM_ID','TEAM_NAME']]
vis_teams.rename(columns={'TEAM_ID':'VISITOR_TEAM_ID'},inplace=True)
new_games = pd.merge(games,home_teams,on='HOME_TEAM_ID',how='left')
new_games = pd.merge(new_games,vis_teams,on='VISITOR_TEAM_ID',how='left')

<ipython-input-36-c5bba020f65b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  home_teams.rename(columns={'TEAM_ID':'HOME_TEAM_ID'},inplace=True)
<ipython-input-36-c5bba020f65b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vis_teams.rename(columns={'TEAM_ID':'VISITOR_TEAM_ID'},inplace=True)


In [38]:
new_games.rename(columns={'TEAM_NAME_x':'HOME_TEAM_NAME', 'TEAM_NAME_y': "VISITOR_TEAM_NAME"},inplace=True)

In [39]:
new_games['year_month'] = new_games['GAME_DATE_EST'].str[:7]

In [31]:
teams['TEAM_NAME'] = teams[["CITY","NICKNAME"]].apply(lambda x: ''.join(x), axis=1)

In [41]:
# new_games_2 = pd.merge(new_games,test,on='year_month',how='left')

In [56]:
result_df = pd.concat([train,test]).drop_duplicates(subset=['group_name', 'year_month'])

In [57]:
result_df.loc[result_df['group_name'] == "AtlantaHawks"] #works

,Unnamed: 0,group_name,year_month,negative %,neutral %,positive %
0,0,AtlantaHawks,2020-07,9.661355,67.231076,23.107570
1,1,AtlantaHawks,2020-08,11.016949,58.474576,30.508475
2,2,AtlantaHawks,2020-09,2.272727,70.454545,27.272727
1,1,AtlantaHawks,2020-10,4.081633,44.897959,51.020408


In [61]:
result_df = result_df[['group_name', 'year_month', 'negative %', 'neutral %',
       'positive %']]

In [90]:
home_teams = result_df
home_teams.rename(columns={'group_name':'HOME_TEAM_NAME'},inplace=True)

In [68]:
home_teams.shape

(116, 5)

In [70]:
home_teams.loc[home_teams['HOME_TEAM_NAME'] == "LosAngelesLakers"]

,HOME_TEAM_NAME,year_month,negative %,neutral %,positive %
39,LosAngelesLakers,2020-07,3.239625,77.135207,19.625167
40,LosAngelesLakers,2020-08,9.027778,61.226852,29.745370
41,LosAngelesLakers,2020-09,8.244681,65.957447,25.797872
27,LosAngelesLakers,2020-10,8.975040,43.547531,47.477430


In [73]:
new_games.shape

(26651, 22)

In [74]:
new_games_2.shape

(26651, 25)

In [85]:
new_games.loc[new_games['year_month'] == "2020-10"].shape

(5, 22)

In [84]:
new_games.loc[new_games['GAME_DATE_EST'].str[:7] == "2020-10"].shape

(5, 22)

In [92]:
new_games_2 = pd.merge(new_games,home_teams,on=['year_month', 'HOME_TEAM_NAME'],how='left')
visitor_teams = result_df[:]
visitor_teams.rename(columns={'HOME_TEAM_NAME':'VISITOR_TEAM_NAME'},inplace=True)
new_games_2 = pd.merge(new_games_2,visitor_teams,on=['year_month', 'VISITOR_TEAM_NAME'],how='left')

In [94]:
new_games_2.loc[new_games_2['year_month'] == "2020-07"].shape #works!

(41, 28)

In [97]:
to_impute = ['negative %_x',
       'neutral %_x', 'positive %_x', 'negative %_y', 'neutral %_y',
       'positive %_y']
for col in to_impute:
  new_games_2[col] = new_games_2[col].fillna(new_games_2[col].mean())

In [98]:
new_games_2

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,...,HOME_TEAM_WINS,HOME_TEAM_NAME,VISITOR_TEAM_NAME,year_month,negative %_x,neutral %_x,positive %_x,negative %_y,neutral %_y,positive %_y
0,2022-12-22,22200477,1610612740,1610612759,2022,126.0,0.484,0.926,0.382,25.0,...,1,New OrleansPelicans,San AntonioSpurs,2022-12,13.760926,53.310051,32.929024,13.85407,53.055657,33.090273
1,2022-12-22,22200478,1610612762,1610612764,2022,120.0,0.488,0.952,0.457,16.0,...,1,UtahJazz,WashingtonWizards,2022-12,13.760926,53.310051,32.929024,13.85407,53.055657,33.090273
2,2022-12-21,22200466,1610612739,1610612749,2022,114.0,0.482,0.786,0.313,22.0,...,1,ClevelandCavaliers,MilwaukeeBucks,2022-12,13.760926,53.310051,32.929024,13.85407,53.055657,33.090273
3,2022-12-21,22200467,1610612755,1610612765,2022,113.0,0.441,0.909,0.297,27.0,...,1,Philadelphia76ers,DetroitPistons,2022-12,13.760926,53.310051,32.929024,13.85407,53.055657,33.090273
4,2022-12-21,22200468,1610612737,1610612741,2022,108.0,0.429,1.000,0.378,22.0,...,0,AtlantaHawks,ChicagoBulls,2022-12,13.760926,53.310051,32.929024,13.85407,53.055657,33.090273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26646,2014-10-06,11400007,1610612737,1610612740,2014,93.0,0.419,0.821,0.421,24.0,...,1,AtlantaHawks,New OrleansPelicans,2014-10,13.760926,53.310051,32.929024,13.85407,53.055657,33.090273
26647,2014-10-06,11400004,1610612741,1610612764,2014,81.0,0.338,0.719,0.381,18.0,...,0,ChicagoBulls,WashingtonWizards,2014-10,13.760926,53.310051,32.929024,13.85407,53.055657,33.090273
26648,2014-10-06,11400005,1610612747,1610612743,2014,98.0,0.448,0.682,0.500,29.0,...,1,Los AngelesLakers,DenverNuggets,2014-10,13.760926,53.310051,32.929024,13.85407,53.055657,33.090273
26649,2014-10-05,11400002,1610612761,1610612758,2014,99.0,0.440,0.771,0.333,21.0,...,1,TorontoRaptors,SacramentoKings,2014-10,13.760926,53.310051,32.929024,13.85407,53.055657,33.090273


In [99]:
new_games_2.to_csv("games_with_sentiment.csv",index=False)

In [17]:
def data_transformation(games,TEAM_ID,YEAR):
  df = games.loc[((games["HOME_TEAM_ID"] == TEAM_ID)
  | (games["VISITOR_TEAM_ID"] == TEAM_ID))].loc[games['SEASON'] == YEAR]
  df.sort_values(by=['GAME_DATE_EST'],ascending=True,inplace=True)
  df = df[['HOME_TEAM_ID', 'VISITOR_TEAM_ID',
                                              'PTS_home', 'FG_PCT_home',
                                              'FT_PCT_home', 'FG3_PCT_home',
                                              'AST_home', 'REB_home',
                                              'TEAM_ID_away', 'PTS_away',
                                              'FG_PCT_away', 'FT_PCT_away',
                                              'FG3_PCT_away', 'AST_away',
                                              'REB_away', 'HOME_TEAM_WINS']]

  df['OTHER_TEAM_ID'] = np.where(df['HOME_TEAM_ID'] == TEAM_ID,
                                df['VISITOR_TEAM_ID'], df['HOME_TEAM_ID'])
  df['WIN'] = np.where(
      (df['HOME_TEAM_ID'] == TEAM_ID) & (df['HOME_TEAM_WINS'] == 1) |
      (df['HOME_TEAM_ID'] != TEAM_ID) & (df['HOME_TEAM_WINS'] == 0), 1, 0
  )
  # Identify columns with 'home' and 'away' suffixes
  home_columns = df.columns[df.columns.str.endswith('_home')]
  away_columns = df.columns[df.columns.str.endswith('_away')]
  # Create new columns with 'current' suffixes based on conditions
  df['PTS_current'] = np.where(df['HOME_TEAM_ID'] == TEAM_ID,
                              df['PTS_home'], df['PTS_away'])
  df['FG_PCT_current'] = np.where(df['HOME_TEAM_ID'] == TEAM_ID,
                                  df['FG_PCT_home'], df['FG_PCT_away'])
  df['FT_PCT_current'] = np.where(df['HOME_TEAM_ID'] == TEAM_ID,
                                  df['FT_PCT_home'], df['FT_PCT_away'])
  df['FG3_PCT_current'] = np.where(df['HOME_TEAM_ID'] == TEAM_ID,
                                  df['FG3_PCT_home'], df['FG3_PCT_away'])
  df['AST_current'] = np.where(df['HOME_TEAM_ID'] == TEAM_ID,
                              df['AST_home'], df['AST_away'])
  df['REB_current'] = np.where(df['HOME_TEAM_ID'] == TEAM_ID,
                              df['REB_home'], df['REB_away'])
  df['PTS_opposing'] = np.where(df['HOME_TEAM_ID'] != TEAM_ID,
                                df['PTS_home'], df['PTS_away'])
  df['FG_PCT_opposing'] = np.where(df['HOME_TEAM_ID'] != TEAM_ID,
                                  df['FG_PCT_home'], df['FG_PCT_away'])
  df['FT_PCT_opposing'] = np.where(df['HOME_TEAM_ID'] != TEAM_ID,
                                  df['FT_PCT_home'], df['FT_PCT_away'])
  df['FG3_PCT_opposing'] = np.where(df['HOME_TEAM_ID'] != TEAM_ID,
                                    df['FG3_PCT_home'], df['FG3_PCT_away'])
  df['AST_opposing'] = np.where(df['HOME_TEAM_ID'] != TEAM_ID,
                                df['AST_home'], df['AST_away'])
  df['REB_opposing'] = np.where(df['HOME_TEAM_ID'] != TEAM_ID,
                                df['REB_home'], df['REB_away'])

  df = df.drop(home_columns, axis=1)
  df = df.drop(away_columns, axis=1)
  df.drop(['HOME_TEAM_ID', 'VISITOR_TEAM_ID','HOME_TEAM_WINS'],
          axis=1,inplace=True)
  return df

In [20]:
YEARS = [i for i in range(2003,2023)]
TEAM_IDS = list(teams['TEAM_ID'].unique())
dfs = [data_transformation(games,combo[0],combo[1])for combo in product(TEAM_IDS,YEARS)]